In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import numpy as np
import itertools
import glob

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

from models import *
from datasets import *

import torch.nn as nn
import torch

##### Build Model

Setting hyperparameters (Default)

In [2]:
dim = 64
n_downsample = 2
channels = 3
img_height = 256
img_width = 256
shared_dim = dim * 2 ** n_downsample
input_shape = (channels, img_height, img_width)

In [3]:
shared_E = ResidualBlock(features=shared_dim).cuda()
E1 = Encoder(dim=dim, n_downsample=n_downsample, shared_block=shared_E).cuda()
E2 = Encoder(dim=dim, n_downsample=n_downsample, shared_block=shared_E).cuda()
shared_G = ResidualBlock(features=shared_dim).cuda()
G1 = Generator(dim=dim, n_upsample=n_downsample, shared_block=shared_G).cuda()
G2 = Generator(dim=dim, n_upsample=n_downsample, shared_block=shared_G).cuda()

load trained model

In [4]:
epoch = 180

E1.load_state_dict(torch.load("./checkpoint/E1_%d.pth" % (epoch), map_location='cuda:0')) # 
E2.load_state_dict(torch.load("./checkpoint/E2_%d.pth" % (epoch), map_location='cuda:0'))
G1.load_state_dict(torch.load("./checkpoint/G1_%d.pth" % (epoch), map_location='cuda:0'))
G2.load_state_dict(torch.load("./checkpoint/G2_%d.pth" % (epoch), map_location='cuda:0'))

<All keys matched successfully>

##### Load Data

In [11]:
root = '/data/FLIR_ADAS_v2'

transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC),
    transforms.RandomCrop((img_height, img_width)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataset = ImageDataset(root=root, transforms_=transforms_, unaligned=False, mode='test')
dataloader = DataLoader(dataset, batch_size=5, shuffle=False)

print(dataset.files_A[0])
print(dataset.files_B[0])

/data/FLIR_ADAS_v2/video_rgb_test/data/video-BzZspxAweF8AnKhWK-frame-000745-SSCRtAHcFjphNPczJ.jpg
/data/FLIR_ADAS_v2/video_thermal_test/data/video-4FRnNpmSmwktFJKjg-frame-000745-L6K5SC6fYjHNC8uff.jpg


/home/xieshaoyuan/anaconda3/envs/pointnet/lib/python3.6/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


##### Visualize Image

In [12]:
from torchvision.utils import save_image

save_path = 'images/test/'
if not os.path.isdir(save_path): os.makedirs(save_path)

with torch.no_grad():
    for batches_done, imgs in enumerate(dataloader):
        X1 = imgs["A"].cuda()
        X2 = imgs["B"].cuda()
        _, Z2 = E2(X1)
        fake_X2 = G1(Z2)
        img_sample = torch.cat((X1.data, X2.data, fake_X2.data), 0)
        save_image(img_sample, save_path+"/%s.png" % (batches_done), nrow=5, normalize=True)
    

RuntimeError: Sizes of tensors must match except in dimension 3. Got 304 and 306 (The offending index is 0)